# Using SDBN Click Model To Overcome Position Bias

This section we use the _Simplified Dynamic Bayesian Network_ (SDBN) to overcome the position bias that we saw with direct Click-Through-Rate. We consider the SDBN judgments and how they compare to just the click through rate.

In [18]:
from ltr.sdbn_functions import all_sessions, get_sessions
from aips import fetch_products, render_judged
import pandas
import aips.indexer
aips.indexer.pull_github_repository("signals")
# if using a Jupyter notebook, includue:
%matplotlib inline

In [19]:
sessions = all_sessions()
products = fetch_products(doc_ids=sessions['doc_id'].unique())

def print_dataframe(dataframe):
    pandas.reset_option("all")
    merged = dataframe.merge(products[["upc", "name"]], left_on='doc_id', right_on='upc', how='left')
    print(merged.rename(columns={"upc": "doc_id"}).set_index("doc_id"))

# Listing 11.7

Click models overcome position bias by learning an examine probability on each ranking. SDBN tracks examines relative to the the last click. This code marks last click position per session so we can compute examine probabilities.

In [20]:
#%load -s calculate_examine_probability ../ltr/sdbn_functions.py
def calculate_examine_probability(sessions):
    last_click_per_session = sessions.groupby(
        ["clicked", "sess_id"])["rank"].max()[True]
    sessions["last_click_rank"] = last_click_per_session
    sessions["examined"] = \
      sessions["rank"] <= sessions["last_click_rank"]
    return sessions

In [21]:
sessions = get_sessions("dryer")
probablity = calculate_examine_probability(sessions).loc[3]
print(probablity)

         query  rank        doc_id  clicked  last_click_rank  examined
sess_id                                                               
3        dryer   0.0   12505451713    False              9.0      True
3        dryer   1.0   84691226727    False              9.0      True
3        dryer   2.0  883049066905    False              9.0      True
3        dryer   3.0   48231011396    False              9.0      True
3        dryer   4.0   74108056764    False              9.0      True
3        dryer   5.0   77283045400    False              9.0      True
3        dryer   6.0  783722274422    False              9.0      True
3        dryer   7.0  665331101927    False              9.0      True
3        dryer   8.0   14381196320     True              9.0      True
3        dryer   9.0   74108096487     True              9.0      True
3        dryer  10.0   74108007469    False              9.0     False
3        dryer  11.0   12505525766    False              9.0     False
3     

# Listing 11.8

Aggregate clicks and examine counts

In [22]:
#%load -s calculate_clicked_examined ../ltr/sdbn_functions.py
def calculate_clicked_examined(sessions):
    sessions = calculate_examine_probability(sessions)
    return sessions[sessions["examined"]] \
        .groupby("doc_id")[["clicked", "examined"]].sum()

In [23]:
sessions = get_sessions("dryer")
clicked_examined_data = calculate_clicked_examined(sessions)
print_dataframe(clicked_examined_data)

              clicked  examined  \
doc_id                            
12505451713       355      2707   
12505525766       268       974   
12505527456       110       428   
14381196320       217      1202   
36172950027        97       971   
36725561977       119       572   
36725578241       130       477   
48231011396       166       423   
48231011402       213       818   
74108007469       208       708   
74108056764       273      1791   
74108096487       235      1097   
77283045400       276      1625   
84691226703       408      2015   
84691226727       804      2541   
665331101927      270      1347   
783722274422      288      1498   
856751002097      133       323   
883049066905      286      2138   
883929085118       44       578   

                                                           name  
doc_id                                                           
12505451713     Frigidaire - Semi-Rigid Dryer Vent Kit - Silver  
12505525766         Smart Choic

/tmp/ipykernel_1042/1625057584.py:5: FutureWarning:

data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.

/tmp/ipykernel_1042/1625057584.py:5: FutureWarning:

use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.



# Listing 11.9

We compute a grade - a probability of relevance - by dividing the clicks by examines. This is the kind of dynamic 'click thru rate' of SDBN, that accounts for whether the result was actually seen by users, not just whether it was shown on the screen.

In [24]:
#%load -s calculate_grade ../ltr/sdbn_functions.py
def calculate_grade(sessions):
    sessions = calculate_clicked_examined(sessions)
    sessions["grade"] = sessions["clicked"] / sessions["examined"]
    return sessions.sort_values("grade", ascending=False)

In [25]:
query = "dryer"
sessions = get_sessions(query)
grade_data = calculate_grade(sessions)
print_dataframe(grade_data)

              clicked  examined     grade  \
doc_id                                      
856751002097      133       323  0.411765   
48231011396       166       423  0.392435   
84691226727       804      2541  0.316411   
74108007469       208       708  0.293785   
12505525766       268       974  0.275154   
36725578241       130       477  0.272537   
48231011402       213       818  0.260391   
12505527456       110       428  0.257009   
74108096487       235      1097  0.214221   
36725561977       119       572  0.208042   
84691226703       408      2015  0.202481   
665331101927      270      1347  0.200445   
783722274422      288      1498  0.192256   
14381196320       217      1202  0.180532   
77283045400       276      1625  0.169846   
74108056764       273      1791  0.152429   
883049066905      286      2138  0.133770   
12505451713       355      2707  0.131141   
36172950027        97       971  0.099897   
883929085118       44       578  0.076125   

         

/tmp/ipykernel_1042/1625057584.py:5: FutureWarning:

data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.

/tmp/ipykernel_1042/1625057584.py:5: FutureWarning:

use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.



# Figure 11.6 Source Code

In [26]:
render_judged(products, calculate_grade(sessions), grade_col="grade", label=f"SDBN judgments for q={query}")

# Listing 11.10 Source Code

In [27]:
query = "transformers dark of the moon"
sessions = get_sessions(query)
grade_data = calculate_grade(sessions)
print_dataframe(grade_data)

              clicked  examined     grade  \
doc_id                                      
97360810042       412       642  0.641745   
400192926087       62       129  0.480620   
97363560449        96       243  0.395062   
97363532149        42       130  0.323077   
93624956037        41       154  0.266234   
47875842328       367      1531  0.239713   
47875841420       217       960  0.226042   
25192107191       176      1082  0.162662   
786936817218      118       777  0.151866   
786936817218      118       777  0.151866   
36725235564        41       277  0.148014   
24543701538       182      1232  0.147727   
47875841369        37       251  0.147410   
47875841406        80       626  0.127796   
24543750949        31       313  0.099042   
47875842335        53       681  0.077827   

                                                           name  
doc_id                                                           
97360810042       Transformers: Dark of the Moon - Blu-ra

/tmp/ipykernel_1042/1625057584.py:5: FutureWarning:

data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.

/tmp/ipykernel_1042/1625057584.py:5: FutureWarning:

use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.



# Figure 11.7 Source Code

In [28]:
render_judged(products, calculate_grade(sessions), grade_col="grade", label=f"SDBN judgments for q={query}")

Up next: [Dealing with Low Confidence Situations](3.SDBN-Confidence-Bias.ipynb)